## roster model estimation

To determine the impact each roster position has on team success, we need to examine the quality of players per game and the result of each game. For each roster position, there will be elite players and secondary players per team. Elite players will be assinged a value of 1 whereas secondary players, a value 2.

### import data sets  "play by play goal detail" and "game detail"

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import datetime, time
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from pylab import hist, show
import scipy
import zipfile


pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 200)


In [2]:
pwd

'/Users/stefanostselios/Desktop/nhl_roster_design-master'

In [3]:
d0 = pd.read_csv('season_games.csv', index_col=0)
d1 = pd.read_csv('team_roster_quality_without_goalies.csv', index_col=0)

FileNotFoundError: File b'team_roster_quality_without_goalies.csv' does not exist

In [ ]:
dm = d0.merge(d1, on=['Season', 'GameNumber'], how='left')

- Calculate the difference between player quality per game for all positions with respect to home team ( Home Team - Visitor Team). There are 5 positions and 2 types of player quality. This will give us a total of 10 differenecs. 

In [ ]:
dm = dm[dm['GameNumber'] <= 21230]

In [ ]:
dm.columns

In [ ]:
dm.shape

In [ ]:
dm['VF'] = dm['VF1'] + dm['VF2']
dm['VD'] = dm['VD1'] + dm['VD2']
dm['HF'] = dm['HF1'] + dm['HF2']
dm['HD'] = dm['HD1'] + dm['HD2']

In [ ]:
dm['VF'].value_counts()

In [ ]:
dm['HF'].value_counts()

In [ ]:
dm['VD'].value_counts()

In [ ]:
dm['HD'].value_counts()

In [ ]:
dm = dm[((dm['VF'] == 12) & (dm['HF'] == 12) & (dm['VD'] == 6) & (dm['HD'] == 6))]

## Summary analysis

In [ ]:
dm.describe()

In [ ]:
dm = dm[['Season', 'GameNumber', 'VTeamCode', 'HTeamCode', 'HGF', 'VGF', 'GD','WinTeam',
         'VF1', 'VF2', 'VD1', 'VD2', 
         'HF1', 'HF2', 'HD1', 'HD2']]

In [ ]:
dm['HomeWin'] = dm.apply(lambda x: 1 if x['WinTeam']=='HOME' else 0, axis=1)
dm['DF1'] = dm['HF1'] - dm['VF1']
dm['DF2'] = dm['HF2'] - dm['VF2']
dm['DD1'] = dm['HD1'] - dm['VD1']
dm['DD2'] = dm['HD2'] - dm['VD2']

In [ ]:
dm.groupby(['WinTeam'])['DF1', 'DF2', 'DD1', 'DD2'].describe()

## Mean number of F1, F2, D1 D1 per team

* create a season-team dataframe
  
  ** number of wins/points/winning percentage

### estimate roster model 

- regress home win on the difference in number of home and visitor players by position and quality (predictor variables). Add a constant to the predictors and use OLS. The purpose is to deterimine the impact each roster positin has on home team success.

In [ ]:
y = dm['HomeWin']  
X = sm.add_constant(dm[['DF1', 'DD1']] )

In [ ]:
result = sm.OLS(y, X).fit()
result.summary()

In [ ]:
result.params